In [ ]:
pip install --upgrade PyPDF2


In [ ]:
pip install langchain


In [ ]:
pip install pypdfium2


In [ ]:
pip install openpyxl


In [1]:
from Application.data_cleaning import process_pdfs_and_build_dataframe


In [ ]:
# This code was to split each 100s PDFs into excel files with two columns:
# Name of the document and clean text
"""
from Application.data_cleaning import process_pdfs_and_build_dataframe
pdf_directory ="../Data/Data/PDFs"
document_texts_900_998 = process_pdfs_and_build_dataframe(pdf_directory, 900, 998)
document_texts_900_998.to_excel("../Data/output_batch_900_998.xlsx", index=False)
"""


In [ ]:
# This code was used to merge every excel files into a big one
"""import pandas as pd
import os

# Create a list for each excel files
all_batches = []

excel_directory = "/home/guillaumelewagon/code/Guillaume2126/Document-Retrieval-Chatbot/Data/"

for batch_number in range(1, 11):
    file_name = f"{(batch_number - 1) * 100 + 1}_{batch_number * 100}"

    excel_filename = f"output_batch_{file_name}.xlsx"
    excel_path = os.path.join(excel_directory, excel_filename)

    df_batch = pd.read_excel(excel_path)
    all_batches.append(df_batch)

all_document_texts = pd.concat(all_batches, ignore_index=True)

output_file_path = os.path.join(excel_directory, "output_all_batches.xlsx")
all_document_texts.to_excel(output_file_path, index=False)"""


1- First model

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from Application.data_cleaning import cleaning
import pandas as pd
import os
import joblib

data_path = "../Data/"
df = pd.read_excel(os.path.join(data_path, "output_all_batches.xlsx"))
df["Clean text"].fillna("", inplace=True)

# Vectorize text
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df["Clean text"])

# Save the TfidfVectorizer
vectorizer_path = "../Application/tfidf_vectorizer.joblib"
joblib.dump(tfidf_vectorizer, vectorizer_path)

# Save the matrix TF-IDF
matrix_path = "../Application/tfidf_matrix.joblib"
joblib.dump(tfidf_matrix, matrix_path)

# Function to find the document similar to the request
def get_most_similar_documents(query, tfidf_matrix, tfidf_vectorizer, top_n=5):
    # Clean and vectorize the request
    query = cleaning(query)
    query_vector = tfidf_vectorizer.transform([query])

    # Calculate the similarity between the request and all the documents
    cosine_similarities = linear_kernel(query_vector, tfidf_matrix).flatten()

    # Sort by similarity
    document_indices = cosine_similarities.argsort()[:-top_n-1:-1]

    # Create a DataFrame with the results
    result_df = df.iloc[document_indices].copy()

    # Add a column for similarity scores
    result_df["Similarity Score"] = cosine_similarities[document_indices]

    return result_df


In [20]:
user_query = "How to use an electric oven :"
similar_documents = get_most_similar_documents(user_query, tfidf_matrix, tfidf_vectorizer)

print("Documents corresponding to the request :")
print(similar_documents[["Name of the document", "Clean text", "Similarity Score"]])


Documents corresponding to the request :
    Name of the document                                         Clean text  \
852     30_wall_oven.pdf  ge monogram use care guide wall oven downloade...   
949        139900300.pdf  use care wwwfrigidairecom usa wwwfrigidaireca ...   
149      30_b3007hlb.pdf  user manual bhlb builtin oven user manual down...   
419        316417012.pdf  pn rev c range e control selfcleaning oven cer...   
141           msh28b.pdf  model mshb electric mini oven hob please read ...   

     Similarity Score  
852          0.701334  
949          0.693870  
149          0.663768  
419          0.619903  
141          0.605817  


Other models or methods could be used like Word2Vec, Doc2Vec or BM25Okapi